In [1]:
import requests
from json_to_psql import spark_to_sql
from pyspark.sql import SparkSession
import mysql

In [2]:
# Send GET request to the URL
url = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    loan_data = response.json()  
  
    print("Loan data retrieved successfully!")
    print("Number of loan applications:", len(loan_data))
else:
    print("Failed to retrieve loan data. Status code:", response.status_code)

Loan data retrieved successfully!
Number of loan applications: 511


In [6]:
spark = SparkSession.builder \
    .appName("APItoMySQL") \
    .getOrCreate()

In [7]:
loan_df = spark.createDataFrame(loan_data)

db_name = 'creditcard_capstone'
table_name = 'CDW_SAPP_loan_application'
url = f'jdbc:mysql://localhost:3306/{db_name}'

properties = {
    'user': 'root',
    'password': 'password',
    'driver': 'com.mysql.jdbc.Driver'
}


In [8]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
spark.sql(f"USE {db_name}")

spark_to_sql(loan_df, table_name='cdw', url = url, properties= properties )
spark.stop()